# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [133]:
import requests
from bs4 import BeautifulSoup
import json
import time
import _thread


host = 'https://www.ptt.cc'
cookies = {'over18': '1'}
board_list = ['Stock', 'Gossiping', 'Lifeismoney', 'movie', 'cookclub', 'Finance']


In [134]:
def parse(response):
    article_title = []
    article_URL = []
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(response.url))
        return
    soup = BeautifulSoup(response.text)
    main_list = soup.find('div', class_='bbs-screen')

    # 依序檢查文章列表中的 tag, 遇到分隔線就結束，忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']

        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break

        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            # 如果文章已經被刪除則跳過
            if not a_title or not a_title.has_attr('href'):
                continue
            article_URL.append(host + a_title['href'])
            article_title.append(a_title.text)
            print('Parse article {}'.format(article_title[-1]), article_URL[-1])
    return article_title, article_URL

def parse_article(response):
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(response.url))
        return
    soup = BeautifulSoup(response.text)
    data = {}
    data['url'] = response.url
    for i in soup.find_all(class_='article-metaline'):
        data[i.find(class_='article-meta-tag').text] = i.find(class_='article-meta-value').text
        print(i.find(class_='article-meta-tag').text, ':',  i.find(class_='article-meta-value').text)

    data['context'] = soup.find(id='main-container').text.split('\n')
    print('\n====== text ======\n', soup.find(id='main-container').text.split('\n'))

    print('data: ', data)
    return data

def write2file(jsondata, filepath): 
    with open(filepath,"a") as f:
        json.dump(a,f, ensure_ascii=False)
        print("載入入檔案完成...")

In [182]:
#串連
startTime = time.time()

# article index

for board in board_list:
    url = 'https://www.ptt.cc/bbs/{}/index.html'.format(board)
    article_response = requests.get(url, cookies = cookies)
    article_URLlist = parse(article_response)[1]
    
    # article context
    for article_URL in article_URLlist:
        response = requests.get(article_URL, cookies = cookies)
        article_data = parse_article(response)
        filepath = 'hw035data/' + board + '.json'
        write2file(article_data, filepath)



finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

Parse article Re: [請益] 元石油正二 溢價138% https://www.ptt.cc/bbs/Stock/M.1586326883.A.C2F.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586327064.A.591.html
Parse article Re: [心得] 大盤看法 04/07 更新 https://www.ptt.cc/bbs/Stock/M.1586327415.A.242.html
Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生 https://www.ptt.cc/bbs/Stock/M.1586327913.A.685.html
Parse article [標的] 宏遠 1460 多 https://www.ptt.cc/bbs/Stock/M.1586328314.A.9B4.html
Parse article [標的] 6668 中揚光 空 https://www.ptt.cc/bbs/Stock/M.1586328404.A.989.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586328868.A.889.html
Parse article [心得] 這波和2015年股災真像 https://www.ptt.cc/bbs/Stock/M.1586329092.A.8DF.html
Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生 https://www.ptt.cc/bbs/Stock/M.1586329267.A.398.html
Parse article [其他] 109年04月08日 三大法人買賣金額統計表 https://www.ptt.cc/bbs/Stock/M.1586329504.A.22D.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586329951.A.7B9.html
Parse

作者 : ichitakajoe (joe)
標題 : Re: [心得] 大盤看法 04/07 更新
時間 : Wed Apr  8 14:30:13 2020

====== text ======
 ['', '作者ichitakajoe (joe)看板Stock標題Re: [心得] 大盤看法 04/07 更新時間Wed Apr  8 14:30:13 2020', '※ 引述《civelant (阿痕)》之銘言：', ': 我敢在今天發文就代表沒在怕酸', ': 要嗆要酸隨便', ': 但造謠 不OK喔~', ': 然後發文此時 DJ+500左右 我如果是個看漲說漲看跌說跌的我挑這時發文幹嘛', ': 傻了嗎我', '嗯 發文後V了1% 費半V了3%', '開始被一連串回文', '結果美股第四節下來 有點開心', '', '但是其實只等於昨天台股收盤', '所以根本沒有任何支持轉空 走中期空頭的跡象', ':', ': 然後我剛看了一下早上推文', ': 有人說空軍空軍每天喊空總會喊到贏', ': 不是', ': 這是時間點問題', ': 今天美股趨勢確立後', ': 直接兩週內台股往下1500點你信不信?', '不信 上週回文就說很多了', '已經沒有C波 拉回也不是9000以下', '底部墊高 就算賭到拉回 肉也不多', '疫情影響股市的效應已經不再', '英國首相進ICU都沒影響了', '老話一句 已反應', ': 不信就來對賭啊', ': 好笑咧 敢嗆又不敢點名', ': 1.', ': 請問反彈了這麼多 還可能是空頭?', ': 請自己去回顧2000 2008 中間彈時彈多少 彈的比這次少我刪id', '我跟Cloud大的想法一致', '這波是2015 所以一路嘎', '沒有可靠的拉回幅度補文章被嘎的波段單（但有趣的是作者根本不做波段單）', ': 空頭還是會有反彈', ': 現在是初跌段反彈', ': 判斷理由', ': 第一心態面:現在處於多空激烈交戰時期 第二:老式技術分析 看價量', ': 真的主跌段 是量縮價跌沒人想玩了', ': 現在多空交戰激烈 高點帶量反轉向下 反彈無量但漲多 正是初跌段的特徵', ': 2.', ': 為何我自己極短線操作卻要寫中線操作建議 你在幹嘛? 害人?', ': 我發文從來都是為了分享自己看法 我寫的文章不是

作者 : c0922949774 (lovetd)
標題 : Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生
時間 : Wed Apr  8 14:38:31 2020

====== text ======
 ['', '作者c0922949774 (lovetd)看板Stock標題Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生時間Wed Apr  8 14:38:31 2020', '※ 引述《Raisto (劉大俠)》之銘言：', ': 剛剛看到版友幫忙查詢兆豐金，其APP顯示：', ': 『2886偏貴合理26元』', ': 我直接開賭注一萬，如果明年（2020）除息前跌到 26，我直接贊助一萬給此APP開發者', '。', ': 依照我的判斷，接下來兆豐金會遇到不少賣壓，但是價格不容易跌到 28塊錢以下。再', '來', ': 按照今年的獲利來說，明年配發 1.7左右應該是可預期。', ': 所以反推接下來的走勢，', ': 1. 年底前，接近填息（31塊錢），會遇上賣壓。', ': 2. 年初一月十號，金控 2019年獲利出爐，開始漲。', ': 3. 董事會股利政策出爐，除權前過前高 32，此價格會遇上賣壓。', ': 總之，26塊錢兆豐金？難歐。', '', '', '', '特別挖這篇文章出來沒有要針對誰～', '', '我也不是APP的開發者，所以也不是來跟兆豐王要錢的(誤～)', '', '只是單純想說在股市中有很多不同策略', '', '但是最難的是貫徹你的策略', '', '當初模型跑出價值投資應該是在26元買進', '', '可是股價離超遠又不停上漲，最高還超過33', '', '你忍得住不進去參加大趴踢嗎？', '', '另一方面來說，股價真的跌到26的時候你承受住的住壓力買進嗎？', '', '所以擬訂股市策略不容易', '', '但是要照著策略不受外在影響執行更不容易', '', '跟在股海浮沉的大家共勉之。', '', '', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 114.136.155.83 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586327913.A.685.html', '推 ss7216tw    : 考古學

作者 : Tosca ( )
標題 : Re: [請益] 如果爆發社區感染是否台股會暴跌？
時間 : Wed Apr  8 14:54:26 2020

====== text ======
 ['', '作者Tosca ( )看板Stock標題Re: [請益] 如果爆發社區感染是否台股會暴跌？時間Wed Apr  8 14:54:26 2020', '台灣很難爆發大規模社區感染', '', '一個11歲確診的去過補習班 全部沒事', '一個5歲確診的照樣去幼兒園四天 全部沒事', '一個確診的看護 去過其他醫院照顧其他病人 全部沒事', '一個澳洲音樂家 隔離了接觸者140幾人 全部沒事', '某間請忽略醫院 隔離了醫護總共一百多位 最後才四個確診', '', '相比起來', '台灣的群聚', '全部都是出國旅遊團', '而且都是歐洲團', '日本的反而沒有 大阪回來一個確診 很早就發生', '結果那團大阪團居然才他一個確診', '反而現在整團超過一半確診的 都是歐洲團', '', '說真的 這病毒在台灣的傳染力', '已經被防到比流感還低了', '現在大家又不外出逛街 又戴口罩 又常洗手', '這病毒跟本傳不了幾個人 就自然消失了', '這就是為什麼 台灣一直都在零星社區感染的原因', '太難群聚', '', '台股了話', '現在也是讓人玩全看不懂', '明明四月後外銷訂單幾乎全滅', '然後內需已經被防疫搞垮', '內外都垮 實在看不到樂觀的理由', '', '很多人認為台灣防疫做的很好', '但說真的 經濟衝擊上', '台灣並沒有比歐美好到哪去', '畢竟台灣是外銷為主', '而歐美外銷現在全沒', '所以台灣自己防疫好也沒用', '經濟一定會跟著歐美一起下去', '我最不滿的就是政府只會強調防疫超前佈署', '結果在處理經濟危機上根本是落後佈署', '而且是遠遠落後', '國外的振興規模都是空前絕後的龐大', '一波又一波', '台灣呢?才一兆???', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 114.27.83.64 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586328868.A.889.html', '推 cblade      : 酷碰卷 帶動噴噴 04/0

作者 : hong888 (自自冉冉 崩崩潰潰)
標題 : [心得] 這波和2015年股災真像
時間 : Wed Apr  8 14:58:10 2020

====== text ======
 ['', '作者hong888 (自自冉冉 崩崩潰潰)看板Stock標題[心得] 這波和2015年股災真像時間Wed Apr  8 14:58:10 2020', '※ 引述《hong888 (自自冉冉 崩崩潰潰)》之銘言：', ': 5.本波空頭對股市的殺傷力有三波', ':   (1)第一波:中國停工造成的產業斷鏈', ':   (2)第二波:全球抗疫讓航空.觀光.服務業停擺，消費緊縮與失業潮', ':   (3)第三波:經濟衰退.企業倒閉.衝擊金融體系釀風暴', ':   我們目前處於第二波與第三波中間的空檔', '', '之前看電視也有老師對比過2015年股災的走勢，', '但真的是越看越像', 'https://i.imgur.com/MUsM1sB.jpg', '', '我也找過2000.2008年的圖，', '都沒有相似本波在十年線跌破後有這麼強勁的反彈', '', '參照此圖來預測2020的中級大反彈', '未來台股將持續震盪走高，可以走到5月中', '反彈目標可以上看半年線，哇，可以彈到11000耶!!!', '', '空軍也不要太難過，', '因為反彈結束後會走這樣', '再度跌破10年線!!!', 'https://i.imgur.com/nlwv3aj.jpg', '', '所以9000點以下還有機會再來，到時候再來借信貸壓身家買下去', '來個財富重分配啊!!', '', '如同我上文所寫，這波空頭對股市影響有三波', '我們目前處於第二波與第三波中間的空檔', '當疫情和緩成為全世界的共識後，要小心第三波經濟"超乎預期"衰退的衝擊', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 219.68.82.150 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586329092.A.8DF.html', '推 moonth66    : 像 04/08 14:58', '推 f204137     : +~也是看到11000 04/08 14:58', '推 DustTo

...

作者 : ELclaire (good job)
標題 : [好雷]重看羊男的迷宮
時間 : Wed Apr  8 10:27:57 2020

====== text ======
 ['', '作者ELclaire (good job)看板movie標題[好雷]重看羊男的迷宮時間Wed Apr  8 10:27:57 2020', '幾年前看過了，', '', '雖然劇情已經有點忘記但是有幾幕還是印象深刻，', '', '像是軍官嘴巴被割開縫合的畫面，', '', '運氣不好的我看完電影隔幾年也受了類似的傷真的有夠慘的傷口超難好；', '', '還有導演美術風格一直很吸引我，', '', '想去itunes收地獄怪客兩集可是都不降價；', '', '最後就是我跟我朋友看的兩個人看完對結局想法整個不同，', '', '後來我接受了她的說法。所以這次又重上了就想說再去看一次，', '', '看看自己對結局的觀感是否改變。', '', '', '多圖多雷多連結版', '', 'https://tinyurl.com/r6rttog', '', '', '看完以後覺得，其實怎樣解讀都是可以的，', '', '如果是屬人世界，小女孩是慘死的，', '', '如果是屬神的世界，小女孩已經得救了。', '', '這其實有點聖經的意味在裡面，', '', '人在世界上受了苦難，克服挑戰，最後被神帶回天家。', '', '小女孩在第二個考驗經不起誘惑吃了兩顆葡萄，釀成一些錯誤，', '', '但是他在第三個考驗因為偉大的愛而通過，', '', '這也是很像聖經裡面會有的概念。', '', '只是有人喜歡這樣的解讀，有人不喜歡，就跟哈密瓜的哈味一樣。', '', '', '', '電影的英文名字其實是潘神，', '', '我想導演採用的不是潘神好色的那一面，', '', '而是潘神屬於牧神的那一面吧。', '', '但是電影口白中，羊男稱自己是法翁，', '', '其實潘神跟法翁是有一點點差異的，', '', '法翁是羅馬神話的半人半人精靈，', '', '而潘神是屬於希臘神話的，', '', '但是兩個會被拿來作相桐概念的投射。', '', '', '只是第二次看就會開始比較看一些細節的部份，', '', '發現這部片裡面的人，好像都有他們各自的笨點。', '', '像是反抗軍帶

作者 : MarcW ( )
標題 : [新聞] 日本超人氣漫改電影！　文青男遇上施虐女
時間 : Wed Apr  8 12:14:47 2020

====== text ======
 ['', '作者MarcW ( )看板movie標題[新聞] 日本超人氣漫改電影！\u3000文青男遇上施虐女時間Wed Apr  8 12:14:47 2020', '新聞網址：https://www.setn.com/News.aspx?NewsID=714549', '', '日本超人氣漫改電影！\u3000文青男遇上施虐女上演調教大戲', '', '日本暢銷300萬本超人氣漫畫真人電影化，《惡之華》上演抖M男主角與抖S女主角上演調', '教大戲，由《道別的早晨就用約定之花點綴吧》動漫名編導岡田麿里編劇力作，《愛情人', '形》《娼年》製片操刀，《就是要妳愛上我》井口昇執導，將於4月24日上映。', '', '演員陣容由榮獲第42屆日本電影金像獎話題獎，及新人演員獎的《在咖啡冷掉之前》伊藤', '健太郎，攜手外型十分具辨識度的《殺手餐廳》、《全境失控》玉城蒂娜、《青夏：戀上', '你的30日》秋田汐梨，以及《新參者完結篇：當祈禱落幕時》飯豐萬理江等人所主演。', '', '春日高男（伊藤健太郎 飾）是一位深愛19世紀詩人波特萊爾作品《惡之華》的平凡初中', '生，他偷拿了暗戀女孩放在教室的體育服，然而一切行徑全被同班同學仲村佐和（玉城蒂', '娜 飾）看在眼裡。仲村以此威脅春日，逼迫他訂下近乎賣身的契約，並屢次強迫他做出', '違背心意的事。具有虐待狂性格的女孩仲村，與被虐性格的男孩春日間，將會有什麼樣令', '人超乎想像的發展？', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 123.193.248.56 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/movie/M.1586319291.A.455.html', '→ zw000ooo009 : 超人氣漫改？ 04/08 12:41', '推 kaj1983     : 漫畫的時間是國中到高中，這選角也太成熟了... 04/08 13:13', '→ Smilingface : 日影只能改編漫畫或動畫了.... 04/08 13:24', '推 xvite

作者 : currykukuo (陳菊濕汗)
標題 : [新聞] 楓林網涉侵權 兩男遭送辦
時間 : Wed Apr  8 15:43:48 2020

====== text ======
 ['', '作者currykukuo (陳菊濕汗)看板movie標題[新聞] 楓林網涉侵權 兩男遭送辦時間Wed Apr  8 15:43:48 2020', '楓林網涉侵權 兩男遭送辦', '', '（中央社記者黃麗芸台北8日電）理工碩士的33歲陳男和32歲莊男自103年起設立影視盜版', '網站「楓林網」，透過網路廣告獲取非法利益，侵權金額近新台幣10億元，刑事局接獲美', '國電影協會跨海提告將兩男送辦。（編輯：李亨山）1090408', '', '新聞網址：https://www.cna.com.tw/news/firstnews/202004085011.aspx', '', '心得:', '', '大家要支持正版 可以支持原創又能合法取得 讚!', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 140.119.121.6 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/movie/M.1586331830.A.82D.html', '※ 編輯: currykukuo (140.119.121.6 臺灣), 04/08/2020 15:44:00', '推 firemm666   : 小鴨什麼時候要被告 04/08 15:44', '推 mach1210    : 爽啦 04/08 15:48', '', '', '']
data:  {'url': 'https://www.ptt.cc/bbs/movie/M.1586331830.A.82D.html', '作者': 'currykukuo (陳菊濕汗)', '標題': '[新聞] 楓林網涉侵權 兩男遭送辦', '時間': 'Wed Apr  8 15:43:48 2020', 'context': ['', '作者currykukuo (陳菊濕汗)看板movie標題[新聞] 楓林網涉侵權 兩男遭送辦時間Wed Apr  8 15:43:48 2020', '楓林網涉侵權 兩男遭送辦', '', '（中央社記者黃麗芸台北8日電）理工碩士的33歲陳男和3

作者 : superer (超級人)
標題 : [問題] 砂鍋放微波爐
時間 : Wed Apr  8 14:03:47 2020

====== text ======
 ['', '作者superer (超級人)看板cookclub標題[問題] 砂鍋放微波爐時間Wed Apr  8 14:03:47 2020', '在全聯社團看到有人用微波爐炒青菜覺得很方便，', '原本考慮要買微波專用盒，但又有點怕怕材質的問題', '在愛料理查食譜看到用微波爐的砂鍋炒青菜，去查了一下原來之前在costco買的砂鍋可以', '放微波爐，', '', '請問微波的時候砂鍋的蓋子也可以蓋起來嗎？', '會從砂鍋蓋的那個小洞噴出來嗎？', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 111.240.92.28 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/cookclub/M.1586325829.A.68D.html', '', '', '']
data:  {'url': 'https://www.ptt.cc/bbs/cookclub/M.1586325829.A.68D.html', '作者': 'superer (超級人)', '標題': '[問題] 砂鍋放微波爐', '時間': 'Wed Apr  8 14:03:47 2020', 'context': ['', '作者superer (超級人)看板cookclub標題[問題] 砂鍋放微波爐時間Wed Apr  8 14:03:47 2020', '在全聯社團看到有人用微波爐炒青菜覺得很方便，', '原本考慮要買微波專用盒，但又有點怕怕材質的問題', '在愛料理查食譜看到用微波爐的砂鍋炒青菜，去查了一下原來之前在costco買的砂鍋可以', '放微波爐，', '', '請問微波的時候砂鍋的蓋子也可以蓋起來嗎？', '會從砂鍋蓋的那個小洞噴出來嗎？', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 111.240.92.28 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/cookclub/M.1586325829.A.68D.html', '', '', '']}
載入入檔案完成...
作者 : a

作者 : ioupoiu (他媽的噁心甲)
標題 : Re: [請益] 強化洗錢防治以來有抓到什麼大案嗎?
時間 : Wed Apr  8 10:10:54 2020

====== text ======
 ['', '作者ioupoiu (他媽的噁心甲)看板Finance標題Re: [請益] 強化洗錢防治以來有抓到什麼大案嗎?時間Wed Apr  8 10:10:54 2020', 'https://reurl.cc/R4X4xe', '', '成果如上 做這些不管是表面的實際的 還是無腦的', '', '最重要的就是通過國際評鑑', '', '至於有沒有什麼大案 除了調查局與銀行法遵、洗錢極少數人外', '', '不是你這種咖可以知道的', '', '', '引述《sam80354 (~Q.Q~)》之銘言：', ': 如題 自從台灣政府開始重視AML之後', ': 這幾年無限上綱的洗錢防治政策', ': 想必搞得金融業的朋友們頭很大', ': 寫不完的報表跟KYC(瘋狂加班)', ': 問客戶問題或要資料被罵爆(業務流失)', ': 還有稽核極端似般雞蛋挑骨頭，還有總行各種丟包', ': 還有各種無腦政策丟出來', ': 代表政府真的認為台灣洗錢問題很嚴重', ': 既然如此，經歷了3-4年的努力，真的有抓到什麼大案子嗎?', ': 不然這幾年大家這麼辛苦操勞，至少要有些成果吧?', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 180.217.136.192 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Finance/M.1586311858.A.B7E.html', '→ avant: 網址04/08 10:11', '懶得縮', '※ 編輯: ioupoiu (180.217.136.192 臺灣), 04/08/2020 10:17:16', '→ avant: 漏一截04/08 10:48', '噓 neverli: 貼網址都不會還這麼嗆喔？04/08 11:28', '好啦縮一下', '※ 編輯: ioupoiu (180.217.136.192 臺灣), 04/08/2020 11:45:21', '→ joy2105feh: 靠北 那八個字有謝震武的聲音.... 04/08 13:

In [183]:
#並連


# article index

def multirequests(para_url, cookies, para_board):
    print(para_url)
    response = requests.get(url, cookies = cookies)
    data = parse_article(response)
    print(data)
    filepath = 'hw035data/' + para_board +'_paralle' +'.json'
    write2file(data, filepath)

startTime = time.time()
for board in board_list:
    url = 'https://www.ptt.cc/bbs/{}/index.html'.format(board)
    article_response = requests.get(url, cookies = cookies)
    article_URLlist = parse(article_response)[1]
    
    # article context
    for article_URL in article_URLlist:
        _thread.start_new_thread( multirequests, (article_URL, cookies, board,) )
#         response = requests.get(article_URL, cookies = cookies)
#         article_data = parse_article(response)
#         filepath = 'hw035data/' + board +'_paralle' +'.json'
#         write2file(article_data, filepath)



finishTime = time.time()
print('time:', finishTime - startTime) # 並聯情況的爬蟲所需時間

Parse article Re: [請益] 元石油正二 溢價138% https://www.ptt.cc/bbs/Stock/M.1586326883.A.C2F.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586327064.A.591.html
Parse article Re: [心得] 大盤看法 04/07 更新 https://www.ptt.cc/bbs/Stock/M.1586327415.A.242.html
Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生 https://www.ptt.cc/bbs/Stock/M.1586327913.A.685.html
Parse article [標的] 宏遠 1460 多 https://www.ptt.cc/bbs/Stock/M.1586328314.A.9B4.html
Parse article [標的] 6668 中揚光 空 https://www.ptt.cc/bbs/Stock/M.1586328404.A.989.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586328868.A.889.html
Parse article [心得] 這波和2015年股災真像 https://www.ptt.cc/bbs/Stock/M.1586329092.A.8DF.html
Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生 https://www.ptt.cc/bbs/Stock/M.1586329267.A.398.html
Parse article [其他] 109年04月08日 三大法人買賣金額統計表 https://www.ptt.cc/bbs/Stock/M.1586329504.A.22D.html
Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？ https://www.ptt.cc/bbs/Stock/M.1586329951.A.7B9.html
Parse

Parse article Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋 https://www.ptt.cc/bbs/Gossiping/M.1586331999.A.6DA.html
Parse article Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋 https://www.ptt.cc/bbs/Gossiping/M.1586332095.A.2CB.html
Parse article Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋 https://www.ptt.cc/bbs/Gossiping/M.1586332102.A.8FA.html
Parse article [問卦] 風箏在陰天擱淺怎麼辦？ https://www.ptt.cc/bbs/Gossiping/M.1586332143.A.689.html
Parse article [政治] Re: [政治] 酸中央無法1天1口罩 柯文哲：是我就要負 https://www.ptt.cc/bbs/Gossiping/M.1586332219.A.324.html
Parse article [問卦] 學姐很仔細洗澡，是不是愛上我？ https://www.ptt.cc/bbs/Gossiping/M.1586332226.A.73E.html
Parse article Re: [新聞] 先生曾去廣東工作！無業30多歲女染疫 清 https://www.ptt.cc/bbs/Gossiping/M.1586332253.A.7B2.html
Parse article [問卦] 愛奇藝是合法的咪? https://www.ptt.cc/bbs/Gossiping/M.1586332273.A.D60.html
Parse article Re: [問卦] 台灣人是不是都不懂經濟學? https://www.ptt.cc/bbs/Gossiping/M.1586332277.A.DB5.html
Parse article Re: [新聞] 韓國瑜指罷韓連署偷跑 聲請停止執行 https://www.ptt.cc/bbs/Gossiping/M.1586332277.A.C62.html
Parse article [問卦] 羅志祥有整型嗎 https://

載入入檔案完成...載入入檔案完成...


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '13', '', 'Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋', '', '', 'generally', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 generally 的文章', '', '', ' 4/08', '', '', '', '', '', '', 'Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋', '', '', 'Rrrxddd', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 Rrrxddd 的文章', '', '', ' 4/08', '', '', '', '', '', '', 'Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋', '', '', 'yniori', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 yniori 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[問卦] 風箏在陰天擱淺怎麼辦？', '', '', 's7887177', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 s7887177 的文章', '', '', ' 4/08', '', '', '', '', '1', '', '[政治] Re: [政治] 酸中央無法1天1口罩 柯文哲：是我就要負', '', '', 'holisea175', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 holisea175 的文章', '', '', ' 4/08', '', '', '', '', '8', '', '[問卦] 學姐很仔細洗澡，是不是愛上我？', '', '', 'lamigo520', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 lamigo520 的文章', '', '', ' 4/08', '', '', '', '', '', '', 'Re: [新聞] 先生曾去廣東工

載入入檔案完成...載入入檔案完成...


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '9', '', '[情報] cFosSpeed與City Maps 2Go Pro限時免費', '', '', 'iamdco', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 iamdco 的文章', '', '', ' 4/08', '', '', '', '', '2', '', '\t\t\t', '\t\t\t\t(本文已被刪除) [Dokyusei]', '\t\t\t', '\t\t\t', '', '-', '', '', ' 4/08', '', '', '', '', '5', '', '[情報] 特力屋 HOLA $100折扣碼', '', '', 'fr606419', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 fr606419 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[情報] 豆穌朋團購，最高獎金每月3萬！', '', '', 'b886642', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 b886642 的文章', '', '', ' 4/08', '', '', '', '', '', '', '', '[公告] 省錢板板規 2018.11.12', '', '', 'kadasaki', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 kadasaki 的文章', '', '', ' 6/11', '!', '', '', '', '66', '', '[公告] 違規檢舉區', '', '', 'medama', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 medama 的文章', '', '', '12/19', 'M', '', '', '', '爆', '', '[公告] 未滿1元之集點資訊 請集中在本篇推文', '', '', 'medama', '', '⋯', '', '搜尋同標題文章'


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '9', '', '[情報] cFosSpeed與City Maps 2Go Pro限時免費', '', '', 'iamdco', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 iamdco 的文章', '', '', ' 4/08', '', '', '', '', '2', '', '\t\t\t', '\t\t\t\t(本文已被刪除) [Dokyusei]', '\t\t\t', '\t\t\t', '', '-', '', '', ' 4/08', '', '', '', '', '5', '', '[情報] 特力屋 HOLA $100折扣碼', '', '', 'fr606419', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 fr606419 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[情報] 豆穌朋團購，最高獎金每月3萬！', '', '', 'b886642', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 b886642 的文章', '', '', ' 4/08', '', '', '', '', '', '', '', '[公告] 省錢板板規 2018.11.12', '', '', 'kadasaki', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 kadasaki 的文章', '', '', ' 6/11', '!', '', '', '', '66', '', '[公告] 違規檢舉區', '', '', 'medama', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 medama 的文章', '', '', '12/19', 'M', '', '', '', '爆', '', '[公告] 未滿1元之集點資訊 請集中在本篇推文', '', '', 'medama', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 medama 的文章', 


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '1', '', '[新聞] 金手指龐德女郎 奧妮布萊克曼辭世享壽94歲', '', '', 'Gotham', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 Gotham 的文章', '', '', ' 4/07', '', '', '', '', '6', '', '[新聞] 《政客誠實中》在說謊才能生存的環境卻', '', '', 'dakkk', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 dakkk 的文章', '', '', ' 4/08', '', '', '', '', '2', '', '[新聞] 本來這週上映！《007》導演回應延期是否', '', '', 'CYKONGG', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 CYKONGG 的文章', '', '', ' 4/08', '', '', '', '', '1', '', '[情報] 第 60 屆 安錫動畫影展確定取消舉辦！', '', '', 'qpr322', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 qpr322 的文章', '', '', ' 4/08', '', '', '', '', '36', '', '[新聞] 華裔喜劇演員錢信伊加入《上氣》演員陣容', '', '', 'CYKONGG', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 CYKONGG 的文章', '', '', ' 4/08', '', '', '', '', '2', '', 'Re: [負雷] 犬鳴村', '', '', 'IBIZA', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 IBIZA 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[問片] 有熟黑白影片的神人，尋片名', '', '', 'ai456ai', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 ai456ai 的


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '', '', '[問題] 英國black garlic pickle使用', '', '', 'plice2362', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 plice2362 的文章', '', '', ' 4/08', '', '', '', '', '1', '', '[閒聊] 雙北主婦小群組', '', '', 'Amberwoods', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 Amberwoods 的文章', '', '', ' 4/08', '', '', '', '', '1', '', '[問題] 煮牛肉麵用料', '', '', 'alex123456', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 alex123456 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[食譜] 苦瓜雞丁炒蛋', '', '', 'egyptengland', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 egyptengland 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[問題] 砂鍋放微波爐', '', '', 'superer', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 superer 的文章', '', '', ' 4/08', '', '', '', '', '', '', '[食譜] 培根高麗菜', '', '', 'aimitsuki', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 aimitsuki 的文章', '', '', ' 4/08', '', '', '', '', '', '57', '', '[板務] 查詢食材 別稱 別字 及查詢文章代碼(AID)', '', '', 'ivysky', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 ivysky 的文

Parse article [新聞] 民眾上銀行不戴口罩 全金聯：行員若因此 https://www.ptt.cc/bbs/Finance/M.1586270574.A.241.html
Parse article [閒聊] 量客戶體溫異常怎麼處理 https://www.ptt.cc/bbs/Finance/M.1586270965.A.F3E.html
Parse article Re: [請益] 強化洗錢防治以來有抓到什麼大案嗎? https://www.ptt.cc/bbs/Finance/M.1586311858.A.B7E.html
Parse article 中信暑期實習進度 https://www.ptt.cc/bbs/Finance/M.1586332462.A.CA4.html
Reach the last article
time: 3.003862142562866https://www.ptt.cc/bbs/Finance/M.1586270574.A.241.htmlhttps://www.ptt.cc/bbs/Finance/M.1586311858.A.B7E.html
https://www.ptt.cc/bbs/Finance/M.1586270965.A.F3E.html


https://www.ptt.cc/bbs/Finance/M.1586332462.A.CA4.html

====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '7', '', '[新聞] 民眾上銀行不戴口罩 全金聯：行員若因此', '', '', 'pulato16', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 pulato16 的文章', '', '', ' 4/07', '', '', '', '', '9', '', '[閒聊] 量客戶體溫異常怎麼處理', '', '', 'pulato16', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 pulato16 的文章', '', '', ' 4/07', '', 


====== text ======
 ['', '', '', '', '看板', '精華區', '', '', '最舊', '‹ 上頁', '下頁 ›', '最新', '', '', '', '', '', '', '', '', '', '', '7', '', '[新聞] 民眾上銀行不戴口罩 全金聯：行員若因此', '', '', 'pulato16', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 pulato16 的文章', '', '', ' 4/07', '', '', '', '', '9', '', '[閒聊] 量客戶體溫異常怎麼處理', '', '', 'pulato16', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 pulato16 的文章', '', '', ' 4/07', '', '', '', '', '', '', 'Re: [請益] 強化洗錢防治以來有抓到什麼大案嗎?', '', '', 'ioupoiu', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 ioupoiu 的文章', '', '', ' 4/08', '', '', '', '', '', '', '中信暑期實習進度', '', '', 'Kenniefeliz', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 Kenniefeliz 的文章', '', '', ' 4/08', '', '', '', '', '', '爆', '', '[公告] 請低調徵求互辦業績', '', '', 'neverli', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 neverli 的文章', '', '', ' 5/13', '', '', '', '', '爆', '', '[公告] 銀行考試書籍請於此篇置底文集中推文！', '', '', 'neverli', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 neverli 的文章', '', '', ' 7/05', '', '', '', '', '13', '', '[公告] 板規  (新增2條)', '', '', 'neverli', '', '⋯', '', '搜尋同標題文章', '搜尋看板內 neverli 的

In [177]:
!python3 main.py

2020-04-08 15:54:55 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: hw026)
2020-04-08 15:54:55 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 18.9.0, Python 3.7.0 (default, Oct  2 2018, 09:20:07) - [Clang 10.0.0 (clang-1000.11.45.2)], pyOpenSSL 17.0.0 (OpenSSL 1.1.0j  20 Nov 2018), cryptography 2.4.2, Platform Darwin-17.7.0-x86_64-i386-64bit
2020-04-08 15:54:55 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-08 15:54:55 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'hw026',
 'NEWSPIDER_MODULE': 'hw026.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['hw026.spiders']}
2020-04-08 15:54:56 [scrapy.extensions.telnet] INFO: Telnet Password: e42afb33bf2d3d51
2020-04-08 15:54:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2020-04-08 15:54:56 [scrapy.extensions.telnet] INFO: Telnet Password: b697492fb3bab938
2020-04-08 15:54:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-04-08 15:54:56 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.do

2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [心得] 大盤看法 04/07 更新
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [標的] 宏遠 1460 多
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [標的] 6668 中揚光 空
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [心得] 這波和2015年股災真像
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [其他] 109年04月08日 三大法人買賣金額統計表
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [請益] 如果爆發社區感染是否台股會暴跌？
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [新聞] 營收：紘康(6457)3月營收9608萬元創新高
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article Re: [請益] 目前預測股市的技術瓶頸在哪個部分?
2020-04-08 15:54:56 [PTTFirst] DEBUG: Parse article [新聞] 疑涉假交易 東貝光電負責人、協理被羈押
2020-0

作者 : livic (.....)
標題 : Re: [請益] 元石油正二 溢價138%
時間 : Wed Apr  8 14:21:20 2020

====== text ======
 ['', '作者livic (.....)看板Stock標題Re: [請益] 元石油正二 溢價138%時間Wed Apr  8 14:21:20 2020', '※ 引述《jiunjie61 (Jiunjie)》之銘言：', ': 請問先進這檔，元石油正2怎麼這麼多人買，溢價138%也不怕，真的是賭明天的油國會議嗎？那萬一談判破局怎辦呢？目前淨值約1.76，按照金管會新的規則，什麼時候會下市?', ': http://i.imgur.com/kooXYt9.jpg', ': 籌碼看起來都在散戶', ': http://i.imgur.com/fN1NBv0.jpg', ': -----', '金管會已經把規則改了', '從之前3天平均淨值小於2改成30天平均淨值小於2才需清算下市，', '且還有一個但書', '"惟因國內外經濟金融情勢變化，致本基金淨資產價值發生重大波動，', '得報金管會核准於一定期間內不受前揭規定限制。"', '上面這條就是霸王條款，只要申請就可以一直在留在市場上交易', '油2這麼大隻的金雞母，元大才不想讓他清算下市。', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 36.225.16.173 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586326883.A.C2F.html', '推 DustToDust  : 金管會已經擺明就是要凹到底 04/08 14:21', '推 YLTYY       : 我就是規定，我說的算 04/08 14:22', '推 clotilde    : 不會下市的樂透! 04/08 14:23', '推 tamama000   : 難看 04/08 14:23', '→ YLTYY       : 484有大咖被慘套啊？ 04/08 14:24', '推 bjk         : 請問一下這個看盤app是什麼啊?謝謝 04/08 14:24', '→ YLTYY       : 籌碼K吧 04/08 14:24', '推 kadolon

作者 : IBIZA (溫一壺月光作酒)
標題 : Re: [負雷] 犬鳴村
時間 : Wed Apr  8 08:55:01 2020

====== text ======
 ['', '作者IBIZA (溫一壺月光作酒)看板movie標題Re: [負雷] 犬鳴村時間Wed Apr  8 08:55:01 2020', '※ 引述《dohow (都好)》之銘言：', ': 幹他媽的尷尬到絕頂升天', ': 接著有一段女主角三吉彩花追尋自己家族的身世跟犬鳴村的真相（日本恐怖片很喜歡這種路線）', ': 女主角想起從小就看到有一個男人的幽靈會徘徊在外婆跟自己的身邊', ': 然後女主角一想起來，那個男幽靈就出現在她背後了', ': 來了，尷尬的地方來了！', ': 就是這裡！', ': 大家心裡準備一下', ': 女主角發現男幽靈出現，她也不知道哪來的靈感，竟然伸手去抓男幽靈的手！', ': 可能想確認對方是幽靈還是人吧？', ': 但是還真的讓她抓到男幽靈的手！ 哈  哈茲咖西！>\\\\\\<', ': 然後到這裡一定讓人覺得，欸幹，所以他是人喔？', ': 沒有，他是鬼', ': 對啊，他表明他是鬼', ': 為什麼鬼的手會被抓住？因為三吉彩花很漂亮吧？', '', '那個不是幽靈啦', '', '在三吉小時候跟她外婆一起見到的, 她外婆說其他人看不見的那個, 的確是幽靈', '那個就是她外婆的爸爸, 三吉的阿祖', '', '但是出現在長大的三吉面前, 讓她抓住手的這個, 是真人', '她阿祖是穿越時空到現在見三吉', '目的是要揭發犬鳴村的真相, 並且讓三吉把剛出生的外婆救出犬鳴村', '', '電影在這邊故意讓三吉彩花抓住她的手', '就是要告訴觀眾, 這個是真人, 不是幽靈, 怎麼連這個都看不懂？', '', '淹死人的電話亭是個時光機, 半夜兩點會自己響', '響著時候接起來, 會有人叫你快去找他', '如果你回答馬上去, 就可以穿越時空回到昭和二十二年的犬鳴村', '如果你一直沒說要去, 就會像三個男生一樣淹死在電話亭裡', '', ': 總之，男幽靈跟女主角說，他知道哪裡可以讓女主角知道犬鳴村當年的真相', ': 接著畫面一轉，女主角捧著一盒膠卷，就是以前電影播放器的那種膠卷', ': 而且膠卷盒上面還貼著犬鳴村的字條，很體貼細心', ': 所以

...

作者 : trex995 (Trex)
標題 : [標的] 6668 中揚光 空
時間 : Wed Apr  8 14:46:42 2020

====== text ======
 ['', '作者trex995 (Trex)看板Stock標題[標的] 6668 中揚光 空時間Wed Apr  8 14:46:42 2020', '1. 標的：中揚光', '', '2. 分類：空', '', '3. 分析/正文：', '目前大盤萬點，板上做多如雨後春筍，我來替空軍加油打氣！', '大盤3/12-3/13有一個大缺口，今天平了3/13的10128的水位', '如果能過突破來到104xx，我也會先停損。', 'https://i.imgur.com/9If5d3m.jpg', '', '', '(1)中揚光2017年年底上市過後，股價直直落，上面屍體真的太多了，', '一拉高就會出現賣壓。今年要站上季線可能都非常困難。', '', '(2)中揚光的本業為，「光學元件」及「光學模具」。', '中揚光從2018到2019年好幾篇新聞都說的三項：車用、投影機及無人機。', '這三項一定是這公司高層要說服對外投資的slogan。', '但目前市場的挑戰都不站在這邊。', '', '車用：各國車廠有人停工，有人缺料，有人銷售不佳。', '投影機領域：目前線上上課，work from home趨勢漸增。', '無人機：3/31起台灣無人機法規正式上路！', '', '(3)中揚光2018及2019年配息表現極差，殖利率不到2%。', '若非長期持有的大股東，此檔股票表現算是不及格。', '台灣光學股偏妖，除了大立光一枝獨秀，其餘不外乎都是炒炒題材消息', '莫忘2018年揚明光/佳凌等表現。', '', '(4)中揚光模具營利主要靠賣中國與韓系，基本上確實跟水果無太大關係。', '但是目前市場相關需求暫時不會太高，相信今年營收表現要要上衝不會太容易', '', '(5)目前計畫斥2億元擴充模具廠產能，也目前受新冠肺炎疫情蔓延影響，', '中揚光在密切觀察投入時點。', '', '(6)中揚光的EPS已經連兩年衰退了，今年加上疫情衝擊，', '衰退預計連三年的機率應該不低。', 'https://i.imgur.com/s35whKE.png', '', '(7)技術指標KD/RS

作者 : ichitakajoe (joe)
標題 : Re: [心得] 大盤看法 04/07 更新
時間 : Wed Apr  8 14:30:13 2020

====== text ======
 ['', '作者ichitakajoe (joe)看板Stock標題Re: [心得] 大盤看法 04/07 更新時間Wed Apr  8 14:30:13 2020', '※ 引述《civelant (阿痕)》之銘言：', ': 我敢在今天發文就代表沒在怕酸', ': 要嗆要酸隨便', ': 但造謠 不OK喔~', ': 然後發文此時 DJ+500左右 我如果是個看漲說漲看跌說跌的我挑這時發文幹嘛', ': 傻了嗎我', '嗯 發文後V了1% 費半V了3%', '開始被一連串回文', '結果美股第四節下來 有點開心', '', '但是其實只等於昨天台股收盤', '所以根本沒有任何支持轉空 走中期空頭的跡象', ':', ': 然後我剛看了一下早上推文', ': 有人說空軍空軍每天喊空總會喊到贏', ': 不是', ': 這是時間點問題', ': 今天美股趨勢確立後', ': 直接兩週內台股往下1500點你信不信?', '不信 上週回文就說很多了', '已經沒有C波 拉回也不是9000以下', '底部墊高 就算賭到拉回 肉也不多', '疫情影響股市的效應已經不再', '英國首相進ICU都沒影響了', '老話一句 已反應', ': 不信就來對賭啊', ': 好笑咧 敢嗆又不敢點名', ': 1.', ': 請問反彈了這麼多 還可能是空頭?', ': 請自己去回顧2000 2008 中間彈時彈多少 彈的比這次少我刪id', '我跟Cloud大的想法一致', '這波是2015 所以一路嘎', '沒有可靠的拉回幅度補文章被嘎的波段單（但有趣的是作者根本不做波段單）', ': 空頭還是會有反彈', ': 現在是初跌段反彈', ': 判斷理由', ': 第一心態面:現在處於多空激烈交戰時期 第二:老式技術分析 看價量', ': 真的主跌段 是量縮價跌沒人想玩了', ': 現在多空交戰激烈 高點帶量反轉向下 反彈無量但漲多 正是初跌段的特徵', ': 2.', ': 為何我自己極短線操作卻要寫中線操作建議 你在幹嘛? 害人?', ': 我發文從來都是為了分享自己看法 我寫的文章不是

作者 : Rrrxddd (有噴 鋼鐵LULU粉 RR)
標題 : Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋
時間 : Wed Apr  8 15:48:13 2020

====== text ======
 ['', '作者Rrrxddd (有噴 鋼鐵LULU粉 RR)看板Gossiping標題Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋時間Wed Apr  8 15:48:13 2020', '', '', '餓死抬頭', '', '嗚嗚嗚', '', '平常相信科學的柯學家', '有了新柯學就不科學了', '', '違背柯學的科學都不行', '阿北在救台灣你懂不懂', '', '給你參考', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223.137.85.123 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1586332095.A.2CB.html', '', '', '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。', '']
data:  {'context': ['',
             '作者Rrrxddd (有噴 鋼鐵LULU粉 RR)看板Gossiping標題Re: [政治] '
             '電鍋殺菌法可行？柯文哲：你會用煮飯電鍋時間Wed Apr  8 15:48:13 2020',
             '',
             '',
             '餓死抬頭',
             '',
             '嗚嗚嗚',
             '',
             '平常相信科學的柯學家',
             '有了新柯學就不科學了',
             '',
             '違背柯學的科學都不行',
             '阿北在救台灣你懂不懂',
             '',
             '給你參考',
             '',
             '--',
             '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 223

作者 : Amberwoods (Amberwoods)
標題 : [閒聊] 雙北主婦小群組
時間 : Wed Apr  8 11:16:24 2020

====== text ======
 ['', '作者Amberwoods (Amberwoods)看板cookclub標題[閒聊] 雙北主婦小群組時間Wed Apr  8 11:16:24 2020', '', '', '自己很喜歡在廚房消磨時光', '也熱愛購買料理用具', '但最近對煮飯有點失去熱情', '可能是太常煮了缺乏動力', '希望可以徵一個煮婦們的小群組（約10-15人）', '', '', '居住區域：雙北', '全職媽或職業媽，只要能時常分享討論都可以（限媽媽就對了）', '', '希望妳是：', '喜歡烹飪', '喜歡孩子享用純天然健康手作的料理', '（偶爾討論不健康的也可以，媽媽需要放鬆）', '喜歡添購廚房物品或交流資訊都歡迎妳', '', '小群組人數少不能潛水，會不定期刪人！', '除了柴米油鹽醬醋茶外，不要討論家務事喔！', '但熟了的話歡迎大家交流興趣與愛好', '', '意者請站內信', '希望能簡略介紹一下自己。', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 61.230.2.49 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/cookclub/M.1586315786.A.821.html', '', '', '']
data:  {'context': ['',
             '作者Amberwoods (Amberwoods)看板cookclub標題[閒聊] 雙北主婦小群組時間Wed Apr  8 '
             '11:16:24 2020',
             '',
             '',
             '自己很喜歡在廚房消磨時光',
             '也熱愛購買料理用具',
             '但最近對煮飯有點失去熱情',
             '可能是太常煮了缺乏動力',
             '希望可以徵一個煮婦們的小群組（約10-15人）',
             '',
             ''

2020-04-08 15:55:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ptt.cc/bbs/cookclub/M.1586319140.A.C77.html> (referer: https://www.ptt.cc/bbs/cookclub/index.html)
作者 : egyptengland (Eva)
標題 : [食譜] 苦瓜雞丁炒蛋
時間 : Wed Apr  8 12:12:16 2020

====== text ======
 ['', '作者egyptengland (Eva)看板cookclub標題[食譜] 苦瓜雞丁炒蛋時間Wed Apr  8 12:12:16 2020', '苦瓜富含維生素C、B1、B2、葉酸及鈣、鎂、鉀等礦物質；而山苦瓜營養價值更高，鉀含', '量是一般苦瓜的1.8倍、鈣質是2.3倍，鎂是2.4倍哦。', '', '將苦瓜和富含蛋白質的雞胸肉及雞蛋一起烹煮，就是一道簡易又營養的便當料理了。', '', 'https://www.instagram.com/p/B-su2Urlh_Q/', 'https://imgur.com/a/G1WX2Oe', '', '食材', '', '１．雞胸肉－－－＞３００ｇ', '２．苦瓜－－－－＞１條', '３．雞蛋－－－－＞２顆', '４．蒜末－－－－＞２瓣量', '', '調味料', '', '１．醬油－-－－---＞１大匙', '２．七味粉－－－＞隨喜', '', '醃料', '', '１．米酒－－－－＞２大匙', '２．味醂－－－－－＞１大匙', '３．白胡椒粉－－－＞１／２茶匙', '４．鹽巴－－－－－＞１／４茶匙', '', '步驟', '', '１．雞胸肉切２cm左右易入口大小，加入醃料抓一下靜置冰箱過夜，', '    或至少靜置１０分鐘。', '', '    https://imgur.com/a/ubPP5fL', '２．苦瓜對半切，去籽去內膜後切成條，煮一鍋滾水，水滾後苦瓜下鍋加少許鹽（食譜份', '量外）燙一分鐘，後起鍋瀝乾備用。', '    https://imgur.com/a/xIs4xdG', '３．雞蛋打成蛋液，熱鍋後加入少許油，蛋液下鍋炒至金黃後起鍋備用。', 

In [186]:
ls

collect_data/ hw026/        main.py       scrapy.cfg


In [187]:
!python3 main.py

2020-04-08 15:58:35 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: hw026)
2020-04-08 15:58:35 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 18.9.0, Python 3.7.0 (default, Oct  2 2018, 09:20:07) - [Clang 10.0.0 (clang-1000.11.45.2)], pyOpenSSL 17.0.0 (OpenSSL 1.1.0j  20 Nov 2018), cryptography 2.4.2, Platform Darwin-17.7.0-x86_64-i386-64bit
2020-04-08 15:58:35 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-08 15:58:35 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'hw026',
 'NEWSPIDER_MODULE': 'hw026.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['hw026.spiders']}
2020-04-08 15:58:35 [scrapy.extensions.telnet] INFO: Telnet Password: 6a7f5bb9f885d663
2020-04-08 15:58:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2020-04-08 15:58:35 [scrapy.core.engine] INFO: Spider opened
2020-04-08 15:58:35 [PTTFirst] DEBUG: dir_path: /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data
2020-04-08 15:58:35 [PTTFirst] DEBUG: runtime_file_path: /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data/Finance.tmp.json.swp
2020-04-08 15:58:35 [PTTFirst] DEBUG: Create temp file for store JSON - /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data/Finance.tmp.json.swp
2020-04-08 15:58:35 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2020-04-08 15:58:35 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6028
2020-04-08 15:58:35 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.ptt.cc/robots.txt> (referer: None)
2020-04-08 15:58:35 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.ptt.cc/robots.txt> (referer: None)
2020-04-08 15:58:35 [scrapy.cor

2020-04-08 15:58:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.ptt.cc/bbs/movie/M.1586274842.A.E1F.html> (referer: https://www.ptt.cc/bbs/movie/index.html)
作者 : turbomons (Τ/taʊ/)
標題 : Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋
時間 : Wed Apr  8 15:57:58 2020

====== text ======
 ['', '作者turbomons (Τ/taʊ/)看板Gossiping標題Re: [政治] 電鍋殺菌法可行？柯文哲：你會用煮飯電鍋時間Wed Apr  8 15:57:58 2020', '※ 引述《sheng612 (無聊的人)》之銘言：', ': 其實...', ': 大家都沒看醫生面對電鍋蒸口罩的表情,', ': 都非常的奧妙,', ': 不過政府既然說有效就有效,', ': 有吃壞東西拉肚子經驗的回家一定要檢查電鍋,', ': 電鍋消毒殺菌效果這麼好,', ': 煮飯吃還會拉肚子.', ': 在這邊檢舉一堆健康節目,', ': 跟你講只要有溫度就會有塑化劑跟亂七八糟的東西,', ': 鼓勵大家自備容器,', ': 看起來塑膠PP非常的健康,', ': 加溫都沒問題.', '', '', '關於這議題', '', '毒物科專家 顏宗海醫師也有出面表示意見', '', '他是覺得用煮食的電鍋去加熱用過的口罩', '', '會有汙染物', '', '怪怪的', '', '還是應該要分開比較安全', '', '另外這方法只適合出入一般場所', '', '不建議出入醫院遭到汙染的口罩也用這種方法消毒', '', '應該拋棄掉不要重復使用', '', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 111.248.193.147 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1586332681.A.EA3.html', '', '', '本網站已依台灣網站內容分級規定處理。此區域為限制級，未滿十八歲者不得瀏覽。', '']
dat

作者 : Gotham (萬惡之城)
標題 : [新聞] 金手指龐德女郎 奧妮布萊克曼辭世享壽94歲
時間 : Tue Apr  7 23:54:00 2020

====== text ======
 ['', '作者Gotham (萬惡之城)看板movie標題[新聞] 金手指龐德女郎 奧妮布萊克曼辭世享壽94歲時間Tue Apr  7 23:54:00 2020', '金手指龐德女郎 奧妮布萊克曼辭世享壽94歲', '', '', '（中央社倫敦6日綜合外電報導）英國老牌女星奧妮布萊克曼（Honor Blackman）曾在007', '情報員詹姆士龐德（James Bond）系列電影「金手指」（Goldfinger）飾演龐德女郎，家', '人今天表示，她已辭世，享壽94歲。', '', '法新社和「衛報」（The Guardian）報導，奧妮布萊克曼在與史恩康納萊（Sean Connery', '）合演的1964年007系列第3部電影「金手指」中，飾演「龐德女郎」蓋羅爾（Pussy', 'Galore），讓當時將近40歲的奧妮布萊克曼聲名大噪。', '', '她的家人今天發表聲明說：「我們很悲傷地宣布，奧妮布萊克曼已辭世，享壽94歲。她在', '薩塞克斯郡（Sussex）路峨斯（Lewes）自宅，因自然原因安詳離世，家人隨侍在側。」', '', '「除了是一位備受敬愛的母親和祖母，她也是一位擁有龐大創意天賦的多產演員。」', '', '龐德電影製片人威爾森（Michael G. Wilson）和布洛克里（Barbara Broccoli）在推特', '發出聲明表示：「一位電影巨星隕落，奧妮布萊克曼在金手指中飾演蓋羅爾的形象，將會', '永留世人心中。」', '', '「她是龐德家族中才華出眾，且備受愛戴的成員，我們的心此時此刻與她的家人同在。」', '', '', '（譯者：吳昇鴻/核稿：劉學源）1090407', '', '', '', '新聞網址：', 'https://www.cna.com.tw/news/amov/202004070390.aspx', '', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 101.137.110.101 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/movi

{'url': 'https://www.ptt.cc/bbs/Stock/M.1586329267.A.398.html', '作者': 'Raisto (劉大俠)', '標題': 'Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生', '時間': 'Wed Apr  8 15:01:06 2020', 'context': ['', '作者Raisto (劉大俠)看板Stock標題Re: [心得] 慘賠六百萬，換一個價值投資APP的誕生時間Wed Apr  8 15:01:06 2020', '※ 引述《c0922949774 (lovetd)》之銘言：', ': ※ 引述《Raisto (劉大俠)》之銘言：', ': : 剛剛看到版友幫忙查詢兆豐金，其APP顯示：', ': : 『2886偏貴合理26元』', ': : 我直接開賭注一萬，如果明年（2020）除息前跌到 26，我直接贊助一萬給此APP開發', '者', ': 。', ': : 依照我的判斷，接下來兆豐金會遇到不少賣壓，但是價格不容易跌到 28塊錢以下。', '再', ': 來', ': : 按照今年的獲利來說，明年配發 1.7左右應該是可預期。', ': : 所以反推接下來的走勢，', ': : 1. 年底前，接近填息（31塊錢），會遇上賣壓。', ': : 2. 年初一月十號，金控 2019年獲利出爐，開始漲。', ': : 3. 董事會股利政策出爐，除權前過前高 32，此價格會遇上賣壓。', ': : 總之，26塊錢兆豐金？難歐。', ': 特別挖這篇文章出來沒有要針對誰～', ': 我也不是APP的開發者，所以也不是來跟兆豐王要錢的(誤～)', ': 只是單純想說在股市中有很多不同策略', ': 但是最難的是貫徹你的策略', ': 當初模型跑出價值投資應該是在26元買進', ': 可是股價離超遠又不停上漲，最高還超過33', ': 你忍得住不進去參加大趴踢嗎？', ': 另一方面來說，股價真的跌到26的時候你承受住的住壓力買進嗎？', ': 所以擬訂股市策略不容易', ': 但是要照著策略不受外在影響執行更不容易', ': 跟在股海浮沉的大家共勉之。', '', '我直接用對帳單來回你', '', 'https://i.imgur.com/HWCozmS.jpg', '', '「逢

...

作者 : MarcW ( )
標題 : [新聞] 日本超人氣漫改電影！　文青男遇上施虐女
時間 : Wed Apr  8 12:14:47 2020

====== text ======
 ['', '作者MarcW ( )看板movie標題[新聞] 日本超人氣漫改電影！\u3000文青男遇上施虐女時間Wed Apr  8 12:14:47 2020', '新聞網址：https://www.setn.com/News.aspx?NewsID=714549', '', '日本超人氣漫改電影！\u3000文青男遇上施虐女上演調教大戲', '', '日本暢銷300萬本超人氣漫畫真人電影化，《惡之華》上演抖M男主角與抖S女主角上演調', '教大戲，由《道別的早晨就用約定之花點綴吧》動漫名編導岡田麿里編劇力作，《愛情人', '形》《娼年》製片操刀，《就是要妳愛上我》井口昇執導，將於4月24日上映。', '', '演員陣容由榮獲第42屆日本電影金像獎話題獎，及新人演員獎的《在咖啡冷掉之前》伊藤', '健太郎，攜手外型十分具辨識度的《殺手餐廳》、《全境失控》玉城蒂娜、《青夏：戀上', '你的30日》秋田汐梨，以及《新參者完結篇：當祈禱落幕時》飯豐萬理江等人所主演。', '', '春日高男（伊藤健太郎 飾）是一位深愛19世紀詩人波特萊爾作品《惡之華》的平凡初中', '生，他偷拿了暗戀女孩放在教室的體育服，然而一切行徑全被同班同學仲村佐和（玉城蒂', '娜 飾）看在眼裡。仲村以此威脅春日，逼迫他訂下近乎賣身的契約，並屢次強迫他做出', '違背心意的事。具有虐待狂性格的女孩仲村，與被虐性格的男孩春日間，將會有什麼樣令', '人超乎想像的發展？', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 123.193.248.56 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/movie/M.1586319291.A.455.html', '→ zw000ooo009 : 超人氣漫改？ 04/08 12:41', '推 kaj1983     : 漫畫的時間是國中到高中，這選角也太成熟了... 04/08 13:13', '→ Smilingface : 日影只能改編漫畫或動畫了.... 04/08 13:24', '推 xvite

作者 : hong888 (自自冉冉 崩崩潰潰)
標題 : [心得] 這波和2015年股災真像
時間 : Wed Apr  8 14:58:10 2020

====== text ======
 ['', '作者hong888 (自自冉冉 崩崩潰潰)看板Stock標題[心得] 這波和2015年股災真像時間Wed Apr  8 14:58:10 2020', '※ 引述《hong888 (自自冉冉 崩崩潰潰)》之銘言：', ': 5.本波空頭對股市的殺傷力有三波', ':   (1)第一波:中國停工造成的產業斷鏈', ':   (2)第二波:全球抗疫讓航空.觀光.服務業停擺，消費緊縮與失業潮', ':   (3)第三波:經濟衰退.企業倒閉.衝擊金融體系釀風暴', ':   我們目前處於第二波與第三波中間的空檔', '', '之前看電視也有老師對比過2015年股災的走勢，', '但真的是越看越像', 'https://i.imgur.com/MUsM1sB.jpg', '', '我也找過2000.2008年的圖，', '都沒有相似本波在十年線跌破後有這麼強勁的反彈', '', '參照此圖來預測2020的中級大反彈', '未來台股將持續震盪走高，可以走到5月中', '反彈目標可以上看半年線，哇，可以彈到11000耶!!!', '', '空軍也不要太難過，', '因為反彈結束後會走這樣', '再度跌破10年線!!!', 'https://i.imgur.com/nlwv3aj.jpg', '', '所以9000點以下還有機會再來，到時候再來借信貸壓身家買下去', '來個財富重分配啊!!', '', '如同我上文所寫，這波空頭對股市影響有三波', '我們目前處於第二波與第三波中間的空檔', '當疫情和緩成為全世界的共識後，要小心第三波經濟"超乎預期"衰退的衝擊', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 219.68.82.150 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586329092.A.8DF.html', '推 moonth66    : 像 04/08 14:58', '推 f204137     : +~也是看到11000 04/08 14:58', '推 DustTo

2020-04-08 15:58:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.ptt.cc/bbs/movie/M.1586275447.A.1C0.html>
{'url': 'https://www.ptt.cc/bbs/movie/M.1586275447.A.1C0.html', '作者': 'dakkk (我是牛我反芻)', '標題': '[新聞] 《政客誠實中》在說謊才能生存的環境卻', '時間': 'Wed Apr  8 00:04:03 2020', 'context': ['', '作者dakkk (我是牛我反芻)看板movie標題[新聞] 《政客誠實中》在說謊才能生存的環境卻時間Wed Apr  8 00:04:03 2020', '新聞網址：https://udn.com/umedia/story/12760/4470768', '', '《政客誠實中》在說謊才能生存的環境卻不能說謊時', '', '《政客誠實中》和不久前的日本片《失憶的總理大臣》〔Hit Me Anyone One More Time', '〕有點異曲同工之妙，《失憶的總理大臣》是讓已當選的總理大臣突然失憶還原成沒有被', '「洗禮」過的單純好人，把政壇搞得一團亂；《政客誠實中》則是讓老練的候選人突然無', '法說謊了。兩部片子都是用不可能發生的狀況，來實現我們可能曾經好奇想過的「如果」', '，如果政客必須要誠實，會是什麼樣子的？雖然不會成真，卻也用假成真的方式來引起觀', '眾反思，有趣又可以想想。', '', '《政客誠實中》雖然沒有到爆笑的程度，但從頭到尾都算是好玩的調性，娛樂性算不錯，', '一般大眾看都沒有問題。', '', '且韓國的選舉和政治雖然和台灣不盡相同，但大部分還相近，至少我們台灣的觀眾是能夠', '理解、感受的。不見得要是平常就很關心政治，只要在選舉時會注意到一些文化（比如候', '選人的電視辯論，記者會的發言，掃街拜票，造勢，挖醜聞爆料等等），就可以看出《政', '客誠實中》的諷刺了；當然，若更常在看新聞，更清楚政治圈的風氣，會看到更多妙處。', '我是覺得它已經是部很夠親民的政治電影。', '', '', '《政客誠實中》的笑點，主要在於在很多該臉不紅氣不喘說出謊話的

作者 : unclefucka (Comic Lo )
標題 : Re: [請益] 如果爆發社區感染是否台股會暴跌？
時間 : Wed Apr  8 14:24:22 2020

====== text ======
 ['', '作者unclefucka (Comic Lo )看板Stock標題Re: [請益] 如果爆發社區感染是否台股會暴跌？時間Wed Apr  8 14:24:22 2020', '※ 引述《Cyborg (廢問只能一篇)》之銘言：', ': 現在歐美都暴發社區感染', ': 股市前陣子也都發生暴跌', ': 但是台股仍然是硬邦邦', ': 是不是台股還沒暴發社區傳染？', ': 如果發生社區傳染是不是暴跌一波？', ': 我覺得清明節後兩週內會有社區傳染', ': 我覺得啦，在想股票要不要售出', ': 目前帳面是賺的', '', '', '   隨著台積電融券迄日前', '', '   1. 每天確診數超棒的，個位數，甚至三個兩個，增長趨勢向下啊!', '', '   2. 又有空談論討伐政敵，然後口罩夠用、準備發送紓困金了呢。', '', '   但', '', '   1. 先要人去玩，然後臭幹一頓，號稱要等14天揭曉。', '', '   2. 倒楣臭小鬼發燒三天仍上學，接觸者聽說七人有點跡象', '', '   3. 連英國首相都準備見佛祖的佛系防範……', '', '   4. 旅外同學說，熱死人的佛羅里達準備是新爆點，現在有不小比例認為會鬧一年', '', '', '   資訊隨人操作啦，不過沒有融資跟上來，空方哪有足夠的肉?', '', '', '--', '※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 202.39.58.9 (臺灣)', '※ 文章網址: https://www.ptt.cc/bbs/Stock/M.1586327064.A.591.html', '噓 lovelydream9: 你整天想放空就放空吧 04/08 14:27', '推 DustToDust  : 被嘎爆 04/08 14:29', '推 mind324     : 海灘那一票很多中標 04/08 14:33', '推 ludewa      : 那應該先作多，等肉多了反手放空呀，賺錢好簡單 04/08 14:35', '', '  

2020-04-08 15:58:37 [scrapy.core.engine] INFO: Closing spider (finished)
2020-04-08 15:58:37 [PTTFirst] DEBUG: self.runtime_file_path - /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data/Stock.tmp.json.swp
2020-04-08 15:58:37 [PTTFirst] DEBUG: self.store_file_path - /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data/Stock-20200408T15:58:37.json
目录为: ['.DS_Store', 'hw026', 'scrapy.cfg', 'main.py', 'collect_data']
CLOSE SPIDER: self.dir_path.exists():  True
CLOSE SPIDER: self.store_file_path.exists():  False
2020-04-08 15:58:37 [PTTFirst] DEBUG: Save result at /Users/karenwang/Desktop/Program/Python/SpiderCourse/PythonHW/hw029/collect_data/Stock-20200408T15:58:37.json
2020-04-08 15:58:37 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 8997,
 'downloader/request_count': 18,
 'downloader/request_method_count/GET': 18,
 'downloader/response_bytes': 92151,
 'downloader/response_count': 18,
 'downloa

# 心得：
改自hw29的程式碼，原本是利用scrapy去各版首頁['Stock', 'Gossiping', 'Lifeismoney', 'movie', 'cookclub', 'Finance']爬個文章，比較三個方式
1. 串連：先找爬版首頁，再將首頁的文章逐一爬出並存檔。時間：22.321188926696777
2. 並聯：先找爬版首頁，個首頁裡的文章並連爬出並存檔。時間：3.003862142562866
3. 用hw29 scrapy方式

In [76]:
import os


# 要檢查的檔案路徑
filepath = "collect_data/Finance-20200329T13:18:24.json"

# 檢查檔案是否存在
if os.path.isfile(filepath):
  print("檔案存在。")
else:
  print("檔案不存在。")

檔案存在。
